In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [4]:
!pip install torch==1.7.1+cu110 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [5]:
!pip install --upgrade gensim mlflow tqdm

Requirement already up-to-date: gensim in /usr/local/lib/python3.7/dist-packages (3.8.3)
Requirement already up-to-date: mlflow in /usr/local/lib/python3.7/dist-packages (1.14.1)
Requirement already up-to-date: tqdm in /usr/local/lib/python3.7/dist-packages (4.59.0)


In [6]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
%%bash

mkdir -p data
#curl -L ./content/drive/My Drive/Colab Notebooks/meli-challenge-2019.tar.bz2 -o ./data/meli-challenge-2019.tar.bz2
#tar jxvf c -C ./data/
curl -L http://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.txt.bz2 -o ./data/SBW-vectors-300-min5.txt.bz2
bunzip2 ./data/SBW-vectors-300-min5.txt.bz2

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   259  100   259    0     0    310      0 --:--:-- --:--:-- --:--:--   310
100  780M  100  780M    0     0  1012k      0  0:13:08  0:13:08 --:--:-- 7584k
bunzip2: Output file ./data/SBW-vectors-300-min5.txt already exists.


In [8]:
%%bash

tar jxvf /content/drive/MyDrive/meli-challenge-2019.tar.bz2

tar (child): /content/drive/MyDrive/meli-challenge-2019.tar.bz2: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


## Librerías

In [9]:
import gzip
import mlflow
import pandas as pd
import tempfile
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from gensim import corpora
from gensim.models import KeyedVectors
from gensim.parsing import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from torch.utils.data import IterableDataset, DataLoader
from tqdm.notebook import tqdm, trange

## Dataset

Nos basaremos en el dataset que creamos para el [notebook 2](./2_datasets.ipynb), con la diferencia de que pasaremos el dataframe de manera directa (esto es para poder hacer split en train/test).

In [10]:
!zcat ./meli-challenge-2019/spanish.train.csv.gz | wc -l

6119101


In [11]:
import numpy as np

train_index = np.random.rand(611910) <= 0.8

with gzip.open("./meli-challenge-2019/spanish.train.csv.gz", "rt") as spanish_corpus,\
    gzip.open("./meli-challenge-2019/spanish-train.csv.gz", "wt") as spanish_train,\
    gzip.open("./meli-challenge-2019/spanish-validation.csv.gz", "wt") as spanish_validation:
  for idx, line in enumerate(tqdm(spanish_corpus, total=611910)):
    if train_index[idx]:
      spanish_train.write(line)
    else:
      spanish_validation.write(line)

  0%|          | 0/611910 [00:00<?, ?it/s]

IndexError: ignored

In [12]:
!zcat ./meli-challenge-2019/spanish-train.csv.gz | wc -l

489944


In [13]:
!zcat ./meli-challenge-2019/spanish-validation.csv.gz | wc -l

121966


In [14]:
!zcat ./meli-challenge-2019/spanish-validation.csv.gz | head

spanish,reliable,Bateria Completa 5 Cuerpos Excelente ,DRUMS
spanish,reliable,Botines Futbol adidas Messi 15.4 Cesped Hombre,FOOTBALL_SHOES
spanish,reliable,Chops Sublimados - Nagual,DRINKING_GLASSES
spanish,reliable,Ventilador De Techo Centrex Madera 4 Palas Con 2 Luces,FANS
spanish,reliable,Equipo De 2 Modulos Marca Pioneer Con Parlantes,MINI_COMPONENT_SYSTEMS
spanish,reliable,Compresor Daihatsu C2050k 2hp (50 Litros),AIR_COMPRESSORS
spanish,reliable,Calefon White Westinhouse,WATER_HEATERS
spanish,reliable,Coche Paraguita De Bebe Nuevo,BABY_STROLLERS
spanish,reliable,"Reloj Caro Cuore Time P/mujer, Acero Y Plástico: Cc13-cebk",WRISTWATCHES
spanish,reliable,Maletín Netbook,LAPTOP_BRIEFCASES


In [16]:
import csv
import random

class MeliChallengeDataset(IterableDataset):
    def __init__(self,
                 dataset_path,
                 dataset_size,
                 random_buffer_size=2048,
                 transform=None):
        assert random_buffer_size > 0
        self.dataset_path = dataset_path
        self.dataset_size = dataset_size
        self.random_buffer_size = random_buffer_size
        self.transform = transform
    
    def __len__(self):
        return self.dataset_size

    def __iter__(self):
        try:
            with gzip.open(self.dataset_path, "rt") as dataset:
                shuffle_buffer = []
                csv_reader = csv.reader(dataset)
                next(csv_reader)

                for line in csv_reader:
                    _, label_quality, title, category = line

                    if self.random_buffer_size == 1:
                        item = {
                            "title": f"{label_quality} {title}",
                            "category": category
                        }
                        if self.transform:
                            item = self.transform(item)
                        yield item
                    else:
                        shuffle_buffer.append({
                            "title": f"{label_quality} {title}",
                            "category": category
                        })

                        if len(shuffle_buffer) == self.random_buffer_size:
                            random.shuffle(shuffle_buffer)
                            for item in shuffle_buffer:
                                if self.transform:
                                    item = self.transform(item)
                                yield item
                            shuffle_buffer = []

            if len(shuffle_buffer) > 0:
                random.shuffle(shuffle_buffer)
                for item in shuffle_buffer:
                    if self.transform:
                        item = self.transform(item)
                    yield item
        except GeneratorExit:
            return

## Preprocesamiento

En este caso vamos a utilizar un sólo módulo para transformar los datos de IMDB. Este se encargará de preprocesar el texto (i.e. normalizarlo) y transformará las palabras en índices de un diccionario para luego poder pasar una secuencia de palabras para buscar en la matriz de embeddings y así permitir mayor manipulación de los embeddings (en lugar de utilizar embeddings fijos).

In [18]:
class RawDataProcessor:
    def __init__(self, 
                 dataset_path,
                 dataset_size,
                 ignore_header=True, 
                 filters=None, 
                 vocab_size=50000):
        if filters:
            self.filters = filters
        else:
            self.filters = [
                lambda s: s.lower(),
                preprocessing.strip_tags,
                preprocessing.strip_punctuation,
                preprocessing.strip_multiple_whitespaces,
                preprocessing.strip_numeric,
                preprocessing.remove_stopwords,
                preprocessing.strip_short,
            ]
        
        # Create dictionary based on all the reviews (with corresponding preprocessing)
        self.dictionary = corpora.Dictionary()
        labels = set()

        with gzip.open(dataset_path, "rt") as dataset:
            csv_reader = csv.reader(dataset)
            if ignore_header:
              next(csv_reader)
            for line in tqdm(csv_reader, total=dataset_size):
                _, label_quality, title, category = line
                self.dictionary.add_documents(
                    [self._preprocess_string(f"{label_quality} {title}")]
                )
                labels.add(category)
        # Filter the dictionary and compactify it (make the indices continous)
        self.dictionary.filter_extremes(no_below=2, no_above=1, keep_n=vocab_size)
        self.dictionary.compactify()
        # Add a couple of special tokens
        self.dictionary.patch_with_special_tokens({
            "[PAD]": 0,
            "[UNK]": 1
        })
        self.idx_to_target = sorted(labels)
        self.target_to_idx = {t: i for i, t in enumerate(self.idx_to_target, start=1)}

    def _preprocess_string(self, string):
        return preprocessing.preprocess_string(string, filters=self.filters)

    def _sentence_to_indices(self, sentence):
        return self.dictionary.doc2idx(sentence, unknown_word_index=1)
    
    def encode_data(self, data):
        return self._sentence_to_indices(self._preprocess_string(data))
    
    def encode_target(self, target):
        return self.target_to_idx.get(target, 0)
    
    def __call__(self, item):
        if isinstance(item, list):
          data = [self.encode_data(i["title"]) for i in item]
          target = [self.encode_target(i["category"]) for i in item]
        else:
          data = self.encode_data(item["title"])
          target = self.encode_target(item["category"])
        
        return {
            "data": data,
            "target": target
        }

## Lectura de datos

En esta ocasión, leeremos los datos de IMDB y lo dividiremos en subconjuntos de entrenamiento y evaluación.

In [19]:
preprocess = RawDataProcessor(
    "./meli-challenge-2019/spanish-train.csv.gz",
    489944
)

  0%|          | 0/489944 [00:00<?, ?it/s]

In [20]:
train_dataset = MeliChallengeDataset(
    "./meli-challenge-2019/spanish-train.csv.gz",
    489944,
    random_buffer_size=2048,
    transform = preprocess
)
validation_dataset = MeliChallengeDataset(
    "./meli-challenge-2019/spanish-validation.csv.gz",
    121966,
    random_buffer_size=1,
    transform=preprocess
)
#test_dataset = MeliChallengeDataset(
 #   "./meli-challenge-2019/spanish.test.csv.gz",
   # 4897903,
 #   random_buffer_size=2048,
  #  transform = preprocess
#)

print(f"Datasets loaded with {len(train_dataset)} training elements and {len(validation_dataset)} validation elements")
print(f"Sample train element:\n{next(iter(train_dataset))}")

Datasets loaded with 489944 training elements and 121966 validation elements
Sample train element:
{'data': [7, 94, 637, 3442, 3441], 'target': 263}


## Collation function

Como en este caso trabajamos con secuencias de palabras (representadas por sus índices en un vocabulario), cuando queremos buscar un *batch* de datos, el `DataLoader` de PyTorch espera que los datos del *batch* tengan la misma dimensión (para poder llevarlos todos a un tensor de dimensión fija). Esto lo podemos lograr mediante el parámetro de `collate_fn`. En particular, esta función se encarga de tomar varios elementos de un `Dataset` y combinarlos de manera que puedan ser devueltos como un tensor de PyTorch. Muchas veces la `collate_fn` que viene por defecto en `DataLoader` sirve (como se vio en el notbook 2), pero este no es el caso. Se define un módulo `PadSequences` que toma un valor mínimo, opcionalmente un valor máximo y un valor de relleno (*pad*) y dada una lista de secuencias, devuelve un tensor con *padding* sobre dichas secuencias.

In [21]:
class PadSequences:
    def __init__(self, pad_value=0, max_length=None, min_length=1):
        assert max_length is None or min_length <= max_length
        self.pad_value = pad_value
        self.max_length = max_length
        self.min_length = min_length

    def __call__(self, items):
        data, target = list(zip(*[(item["data"], item["target"]) for item in items]))
        seq_lengths = [len(d) for d in data]

        if self.max_length:
            max_length = self.max_length
            seq_lengths = [min(self.max_length, l) for l in seq_lengths]
        else:
            max_length = max(self.min_length, max(seq_lengths))

        data = [d[:l] + [self.pad_value] * (max_length - l)
                for d, l in zip(data, seq_lengths)]
            
        return {
            "data": torch.LongTensor(data),
            "target": torch.LongTensor(target)
        }

## DataLoaders

Ya habiendo definido nuestros conjuntos de datos y nuestra `collation_fn`, podemos definir nuestros `DataLoader`s, uno para entrenamiento y otro para evaluación. Ver que la diferencia fundamental está en `shuffle`, no queremos mezclar los valores de evaluación cada vez que evaluamos porque al evaluar mediante *mini-batchs* nos puede generar inconsistencias.

In [40]:
EPOCHS = 6
FILTERS_COUNT = 100
FILTERS_LENGTH = [2, 3, 4]
pad_sequences = PadSequences()
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False,
                          collate_fn=pad_sequences, drop_last=False)
validation_loader = DataLoader(validation_dataset, batch_size=64, shuffle=False,
                               collate_fn=pad_sequences, drop_last=False)
#test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False,
 #                              collate_fn=pad_sequences, drop_last=False)

In [41]:
next(iter(train_loader))

{'data': tensor([[   7,  534, 4732, 4733,  912,  250,  251,    0,    0,    0],
         [   7, 4095, 4094, 4093,    0,    0,    0,    0,    0,    0],
         [   7,  770, 1686,    1,    1, 1688,  961, 1687,    0,    0],
         [   7, 4133, 3222, 1335, 2606, 1697, 3636, 4132,    0,    0],
         [   7,  305, 3907,  124, 1084, 1142,    0,    0,    0,    0],
         [   7, 1067, 1650, 3043, 3044, 3045, 3042,    6,    0,    0],
         [   7, 2216, 1494, 2218, 2217, 2219,    0,    0,    0,    0],
         [   7,  697,   95,  763,  764,  762,    0,    0,    0,    0],
         [   7,  693, 1111,    0,    0,    0,    0,    0,    0,    0],
         [   7,  522,  518,  519,  520,  521,   14,  312,    0,    0],
         [   7,  368,  187,  536,  282,  878,  843,    0,    0,    0],
         [   7, 1659,  197, 1208, 2883,  851,    0,    0,    0,    0],
         [   7, 2034, 2033,  398,  400,  727,    0,    0,    0,    0],
         [   7,  988, 3086, 3084, 2732,  752, 3085,    0,    0,    0]

## El modelo de clasificación

Para clasificación utilizaremos un perceptrón multicapa de dos capas ocultas. Claramente este modelo es naive y prácticamente todo lo que está *hardcodeado* (e.g. los tamaños de las capas o la cantidad de capas) podría ser parte de los parámetros del modelo. En particular, tenemos la capa de `Embeddings` que es rellenada con los valores de embeddings preentrenados (los de Glove en este caso).

In [42]:
!head ./data/SBW-vectors-300-min5.txt 

1000653 300
de -0.029648 0.011336 0.019949 -0.088832 -0.025225 0.056844 0.025473 0.014068 0.163694 -0.067154 0.014738 0.027134 0.066443 -0.044846 -0.044987 -0.040898 0.030311 0.034196 -0.049240 0.008537 -0.068091 -0.087938 0.035300 0.149385 -0.012350 0.012613 0.029350 0.069596 0.039111 0.057652 0.069954 -0.066217 -0.041784 0.028623 0.026772 -0.066392 0.002953 -0.012188 -0.030363 0.040222 0.034858 0.027469 -0.029034 -0.048748 -0.038582 -0.051553 -0.033501 -0.019008 0.003043 0.110712 -0.025096 0.111082 0.035244 0.114207 0.010195 0.051511 -0.040649 -0.113944 0.044873 0.052011 0.067360 0.049054 -0.127085 -0.031846 0.032848 0.040825 -0.084873 0.059801 -0.067424 0.016531 -0.084565 0.057024 0.083288 -0.010136 -0.048508 0.051757 0.046664 0.018102 -0.052320 -0.000765 0.053662 -0.009967 0.082858 0.009068 0.054575 -0.003466 -0.023376 0.023069 0.088513 0.018504 -0.039503 -0.032980 -0.002139 0.000010 -0.107627 0.007699 0.046351 -0.003062 0.030500 0.113650 0.032536 -0.097301 -0.013734 0.098345 0.080

In [43]:
class MeliChallengeClassifier(nn.Module):
    def __init__(self, 
                 pretrained_embeddings_path, 
                 dictionary,
                 n_labels,
                 vector_size,
                 freeze_embedings):
        super().__init__()
        embeddings_matrix = torch.randn(len(dictionary), vector_size)
        embeddings_matrix[0] = torch.zeros(vector_size)
        with open(pretrained_embeddings_path, "rt") as fh:
            next(fh)
            for line in fh:
                word, vector = line.strip().split(None, 1)
                if word in dictionary.token2id:
                    embeddings_matrix[dictionary.token2id[word]] =\
                        torch.FloatTensor([float(n) for n in vector.split()])
        self.embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
                                                       freeze=freeze_embedings,
                                                       padding_idx=0)
        self.convs = []
        for filter_lenght in FILTERS_LENGTH:
            self.convs.append(
                nn.Conv1d(vector_size, FILTERS_COUNT, filter_lenght)
            )
        self.convs = nn.ModuleList(self.convs)
        self.fc = nn.Linear(FILTERS_COUNT * len(FILTERS_LENGTH), 128)
        self.output = nn.Linear(128, n_labels)
        self.vector_size = vector_size
    #Acá hay que reestructurar los datos :) 
    @staticmethod    
    def conv_global_max_pool(x, conv):
        return F.relu(conv(x).transpose(1, 2).max(1)[0])
    
    def forward(self, x):
        x = self.embeddings(x).transpose(1, 2)  # Conv1d takes (batch, channel, seq_len)
        x = [self.conv_global_max_pool(x, conv) for conv in self.convs]
        x = torch.cat(x, dim=1)
        x = F.sigmoid(self.fc(x))
        x = self.output(x)
        return x
        

In [44]:
model = MeliChallengeClassifier("./data/SBW-vectors-300-min5.txt", 
                                preprocess.dictionary, 
                                len(preprocess.target_to_idx) + 1,
                                300, True)

In [45]:
print(model)

MeliChallengeClassifier(
  (embeddings): Embedding(50002, 300, padding_idx=0)
  (convs): ModuleList(
    (0): Conv1d(300, 100, kernel_size=(2,), stride=(1,))
    (1): Conv1d(300, 100, kernel_size=(3,), stride=(1,))
    (2): Conv1d(300, 100, kernel_size=(4,), stride=(1,))
  )
  (fc): Linear(in_features=300, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=633, bias=True)
)


In [46]:
device = torch.device("cuda")
model = model.to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
for epoch in trange(EPOCHS):
  model.train()
  running_loss = []
  for idx, batch in enumerate(tqdm(train_loader)):
    optimizer.zero_grad()
    data = batch["data"].to(device)
    target = batch["target"].to(device)
    output = model(data)
    loss_value = loss(output, target)
    loss_value.backward()
    optimizer.step()
    running_loss.append(loss_value.item())
  print("train_loss", sum(running_loss) / len(running_loss), epoch)  

  model.eval()
  running_loss = []
  targets = []
  predictions = []
  for batch in tqdm(validation_loader):
    data = batch["data"].to(device)
    target = batch["target"].to(device)
    output = model(data)
    running_loss.append(
      loss(output, target).item()
    )
    targets.extend(target.squeeze().cpu().detach().numpy())
    predictions.extend(output.argmax(axis=1).detach().cpu().numpy())
  print("validation_loss", sum(running_loss) / len(running_loss), epoch)
  print("validation_bacc", balanced_accuracy_score(targets, predictions), epoch)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/7656 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


train_loss 3.0836016296593 0


  0%|          | 0/1906 [00:00<?, ?it/s]

...

validation_loss 2.6411592260175087 4
validation_bacc 0.4692982740771822 4


  0%|          | 0/7656 [00:00<?, ?it/s]

train_loss 1.163173335243324 5


  0%|          | 0/1906 [00:00<?, ?it/s]

validation_loss 2.6098430599208142 5
validation_bacc 0.47793330822674407 5


## Experimento de MLflow

Por último, ya tenemos todos los bloques para construir nuestro experimento de MLflow. Anotamos un par de parámetros  (estos pueden ser todos los que se consideren necesarios) y lanzamos a correr nuestro experimento. Cada vez que finaliza un epoch guardamos algunas métricas. Al finalizar todos los epochs corremos algunas métricas extras de evaluación y guardamos algunos datos extra que nos servirán para calcular otras métricas a futuro.

In [50]:
mlflow.set_experiment("a_naive_experiment")

with mlflow.start_run():
    mlflow.log_param("model_name", "mlp")
    mlflow.log_param("freeze_embedding", True)
    mlflow.log_params({
        "embedding_size": 50,
        "hidden1_size": 128,
        "hidden2_size": 128
    })
    model = MeliChallengeClassifier("./data/SBW-vectors-300-min5.txt", 
                                    preprocess.dictionary, 
                                    len(preprocess.target_to_idx) + 1,
                                    300, True)
    device = torch.device("cuda")
    model = model.to(device)
    loss = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    for epoch in trange(3):
        model.train()
        running_loss = 0
        steps=0
        for idx, batch in enumerate(tqdm(train_loader)):
            optimizer.zero_grad()
            data = batch["data"].to(device)
            target = batch["target"].type(torch.LongTensor).to(device)
            output = model(data)
            loss_value = loss(output, target)
            loss_value.backward()
            optimizer.step()
            running_loss +=loss_value.item()
            steps += 1       
        mlflow.log_metric("train loss", running_loss / steps, epoch)
        
        model.eval()
        running_loss = []
        targets = []
        predictions = []
        for batch in tqdm(validation_loader):
            data = batch["data"].to(device)
            target = batch["target"].type(torch.LongTensor).to(device)
            output = model(data)
            running_loss.append(
                loss(output, target).item()
            )
            targets.extend(batch["target"].numpy())
            predictions.extend(output.squeeze().cpu().detach().numpy())
        mlflow.log_metric("validation_loss", sum(running_loss) / len(running_loss), epoch)
        #mlflow.log_metric("validation_bacc", balanced_accuracy_score(targets, predictions), epoch)
                

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/7656 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


  0%|          | 0/1906 [00:00<?, ?it/s]

  0%|          | 0/7656 [00:00<?, ?it/s]

  0%|          | 0/1906 [00:00<?, ?it/s]

  0%|          | 0/7656 [00:00<?, ?it/s]

  0%|          | 0/1906 [00:00<?, ?it/s]

In [51]:
!tar zcvf mlruns.tar.gz mlruns

mlruns/
mlruns/0/
mlruns/0/meta.yaml
mlruns/.trash/
mlruns/1/
mlruns/1/meta.yaml
mlruns/1/9cbad8ae683a4744b139543a2ceadc23/
mlruns/1/9cbad8ae683a4744b139543a2ceadc23/metrics/
mlruns/1/9cbad8ae683a4744b139543a2ceadc23/params/
mlruns/1/9cbad8ae683a4744b139543a2ceadc23/params/embedding_size
mlruns/1/9cbad8ae683a4744b139543a2ceadc23/params/hidden2_size
mlruns/1/9cbad8ae683a4744b139543a2ceadc23/params/hidden1_size
mlruns/1/9cbad8ae683a4744b139543a2ceadc23/params/freeze_embedding
mlruns/1/9cbad8ae683a4744b139543a2ceadc23/params/model_name
mlruns/1/9cbad8ae683a4744b139543a2ceadc23/artifacts/
mlruns/1/9cbad8ae683a4744b139543a2ceadc23/tags/
mlruns/1/9cbad8ae683a4744b139543a2ceadc23/tags/mlflow.source.type
mlruns/1/9cbad8ae683a4744b139543a2ceadc23/tags/mlflow.source.name
mlruns/1/9cbad8ae683a4744b139543a2ceadc23/tags/mlflow.user
mlruns/1/9cbad8ae683a4744b139543a2ceadc23/meta.yaml
mlruns/1/b8fd9dd006c04fbe997052c3a2cbf40e/
mlruns/1/b8fd9dd006c04fbe997052c3a2cbf40e/meta.yaml
mlruns/1/b8fd9dd006c04

In [ ]:
%%bash

# Una vez que lo descargan, lo descomprimen en su máquina con el siguiente comando
tar xvf mlruns.tar.gz

# Luego ejecutan en el lugar que lo descomprimieron lo siguiente
mlflow ui